# APIM ❤️ AI Foundry

## 🎨 Image Generation and multi-modal analysis + Authentication using JWT
![flow](../../images/image-gen.gif)

Playground to try the [OAuth 2.0 authorization feature](https://learn.microsoft.com/azure/api-management/api-management-authenticate-authorize-azure-openai#oauth-20-authorization-using-identity-provider) using identity provider to enable more fine-grained access to AI Foundry models by particular users or client.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`... 


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the models and versions according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

aiservices_config = [{"name": "images", "location": "westus3"}]

models_config = [{"name": "gpt-image-1", "publisher": "OpenAI", "version": "2025-04-15", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "FLUX-1.1-pro", "publisher": "Black Forest Labs", "version": "1", "sku": "GlobalStandard", "capacity": 100},
                 {"name": "gpt-4.1", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 100}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

app_registration_name = f"{deployment_name}-app" # name of the app that will be registered in Microsoft Entra ID

utils.print_ok('Notebook initialized')


<a id='1'></a>
### 1️⃣ Create the App Registration in Microsoft Entra ID
The following command creates a client application registration

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

client_id = None
output = utils.run(f"az ad app list --filter \"displayName eq '{app_registration_name}'\"", f"Retrieved app registration with name {app_registration_name}", "Failed to get the app registration")
if output.success and output.json_data:
    client_id = output.json_data[0]['appId']
else:
    output = utils.run(f"az ad app create --display-name {app_registration_name} --is-fallback-public-client true", f"Created app registration with name {app_registration_name}", "Failed to create the app registration")
    if output.success and output.json_data:
        client_id = output.json_data['appId']

print(f"👉🏻 Client Id: {client_id}")


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.


In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
        "tenantId": { "value": tenant_id },
        "clientId": { "value": client_id }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='4'></a>
### 4️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key

<a id='5'></a>
### 5️⃣ Create a device flow to get the access token

Notes for fine grained authorization:
- The APIM [JWT validation policy](https://learn.microsoft.com/azure/api-management/validate-azure-ad-token-policy) can check for specific claims (that needs to exist in the token) and apply fine-grained authorization.
- Group claims is a typical method. You can use this approach to drive authorization. However, when the user is a member of too many groups, the `groups` will be excluded from the token due to limitations in token size.
- An alternative is to configure app role definitions and assign users/groups to app roles. This Zero Trust developer best practice improves flexibility and control while increasing application security with least privilege. [Learn more](https://learn.microsoft.com/security/zero-trust/develop/configure-tokens-group-claims-app-roles).
- To obtain the `roles` claim, navigate to the "Expose an API" section of the App Registration. Add the Application ID URI and a scope. Then, copy the full scope `(app://<id>/scope)` and replace in the scope array below.
- Navigate to the "App Roles" blade and create an App Role (ex: OpenAI.ChatCompletion) for Users/Groups members. Then assign the testing user or group to the App Role.   
- After logging in, use https://jwt.io/ to decode the `access_token` variable and verify that the `roles` are being sent.
- With the above configuration, you can add the following fragment to the APIM policy to verify that the user belongs to a specific App Role:
```
            <required-claims>
                <claim name="roles" match="any">
                    <value>OpenAI.ChatCompletion</value>
                </claim>
            </required-claims>
```



In [ ]:
import json
import msal

app = msal.PublicClientApplication(client_id, authority = "https://login.microsoftonline.com/" + tenant_id)

flow = app.initiate_device_flow(scopes = ["User.Read"])

if "user_code" not in flow:
    raise ValueError(
        "Fail to create device flow. Err: %s" % json.dumps(flow, indent = 4))

print(flow["message"])

<a id='6'></a>
### 6️⃣ Acquire the token and query the graph API

In [ ]:
import requests, base64, json

result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result['access_token']
    # Calling graph using the access token
    header, payload, signature = access_token.split('.')
    def pad(b): return b + '=' * (-len(b) % 4)
    print("Decoded JWT Header and Payload:")
    print(json.dumps(json.loads(base64.urlsafe_b64decode(pad(header)).decode('utf-8')), indent=4))
    print(json.dumps(json.loads(base64.urlsafe_b64decode(pad(payload)).decode('utf-8')), indent=4))

    graph_data = requests.get(  # Use token to call downstream service
        "https://graph.microsoft.com/v1.0/me",
        headers={'Authorization': 'Bearer ' + access_token},).json()
    print("Graph API call result: %s" % json.dumps(graph_data, indent = 2))
    # print(access_token) # Use a tool like https://jwt.io/ to decode the access token and see its contents
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug



<a id='requests'></a>
### 🧪 Test the API using the access token

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to debug the policy.

In [ ]:
url = f"{apim_resource_gateway_url}/{inference_api_path}/openai/deployments/{models_config[2]['name']}/chat/completions?api-version={inference_api_version}"

messages = { "messages": [
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

response = requests.post(url, headers = {'api-key': api_key, 'Authorization': 'Bearer ' + access_token}, json = messages)
utils.print_response_code(response)

if (response.status_code == 200):
    data = json.loads(response.text)
    print(f"💬 {data.get("choices")[0].get("message").get("content")}")
else:
    print(response.text)


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

### 🎨 Generating an image with a simple prompt using two different models

In [ ]:
from IPython.display import display
from PIL import Image as PILImage
from base64 import b64decode
from io import BytesIO

image_b64 = []

for model in models_config:
    print(f"Generating image using model: {model['name']}")
    
    image_url = f"{apim_resource_gateway_url}/{inference_api_path}/openai/deployments/{model['name']}/images/generations?api-version={inference_api_version}"

    image_payload = {
        "prompt": "A futuristic cityscape at sunset, vibrant colors, high detail",
        "n": 1,
        "size": "1024x1024",
        "output_format": "png"
    }

    image_response = requests.post(
        image_url,
        headers={
            "api-key": api_key,
            "Authorization": "Bearer " + access_token
        },
        json=image_payload
    )

    utils.print_response_code(image_response)

    if image_response.status_code == 200:
        image_data = image_response.json()
        print("Image generation successful!")
        image_b64.append(image_data["data"][0]["b64_json"])
        print("Image Base64 Data:", image_b64[-1][:50])  # Print first 50 characters for brevity
        image=PILImage.open(BytesIO(b64decode(image_b64[-1])))
        ##### Uncomment the line below to save the images generated by each model
        ## image.save(f"{model['name']}_image.png")
        display(image)
    else:
        print(image_response.text)


### 🤔 Process and compare both images

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint = f"{apim_resource_gateway_url}/{inference_api_path}",
    api_key = api_key,
    api_version = inference_api_version
)

response = client.chat.completions.create(
    model = models_config[2]['name'], 
    messages = [
        {"role": "system", "content": "You are a sarcastic helpful assistant."},
        {"role": "user", "content": [
            {"type": "text", "text": "describe these two image, and tell me which one is better"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64[0]}"}},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64[1]}"}}
            ]
        }
    ], 
    extra_headers = {"Authorization": "Bearer " + access_token}
)

print(f"💬 {response.choices[0].message.content}")

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.